In [7]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import py_stringmatching as sm

# Solution Proposed

## 1 - Load Data

In [2]:
# Load the dataset
Xtr0 = pd.read_csv('Xtr0.csv', index_col=0)
Xtr1 = pd.read_csv('Xtr1.csv', index_col=0)
Xtr2 = pd.read_csv('Xtr2.csv',  index_col=0)
# Load the matrix representation of the sequences
Xtr0_mat100 = pd.read_csv('Xtr0_mat100.csv', header=None, sep=' ').values
Xtr1_mat100 = pd.read_csv('Xtr1_mat100.csv', header=None, sep=' ').values
Xtr2_mat100 = pd.read_csv('Xtr2_mat100.csv', header=None, sep=' ').values
# Load the labels
Ytr0 = pd.read_csv('Ytr0.csv', index_col=0)
Ytr1 = pd.read_csv('Ytr1.csv', index_col=0)
Ytr2 = pd.read_csv('Ytr2.csv', index_col=0)
# Convert the labels to -1, 1
Ytr0 = 2*Ytr0['Bound'].values - 1
Ytr1 = 2*Ytr1['Bound'].values - 1
Ytr2 = 2*Ytr2['Bound'].values - 1

## 2 - Compute Kernel Matrix

### 2.1 - Gaussian Kernel

In [369]:
# Define the kernel
def lin_kernel(x, y):
    return np.dot(x, y)

def dist_kernel(x, y):
    return np.linalg.norm(x-y)

def exp_kernel(K_dist, sigma=1):
    return np.exp(-K_dist**2/(2*sigma**2))

In [303]:
# Initialize the distance matrices
K_dist0 = np.zeros((Xtr0_mat100.shape[0], Xtr0_mat100.shape[0]))
K_dist1 = np.zeros((Xtr1_mat100.shape[0], Xtr1_mat100.shape[0]))
K_dist2 = np.zeros((Xtr2_mat100.shape[0], Xtr2_mat100.shape[0]))

# Compute the distance matrices
for i in tqdm(range(Xtr0_mat100.shape[0]), desc="Computing K_dist0"):
    for j in range(Xtr0_mat100.shape[0]):
        K_dist0[i, j] = dist_kernel(Xtr0_mat100[i], Xtr0_mat100[j])

for i in tqdm(range(Xtr1_mat100.shape[0]), desc="Computing K_dist1"):
    for j in range(Xtr1_mat100.shape[0]):
        K_dist1[i, j] = dist_kernel(Xtr1_mat100[i], Xtr1_mat100[j])

for i in tqdm(range(Xtr2_mat100.shape[0]), desc="Computing K_dist2"):
    for j in range(Xtr2_mat100.shape[0]):
        K_dist2[i, j] = dist_kernel(Xtr2_mat100[i], Xtr2_mat100[j])

print(K_dist0.mean(), K_dist1.mean(), K_dist2.mean())
print(K_dist0.std(), K_dist1.std(), K_dist2.std())

Computing K_dist2: 100%|██████████| 2000/2000 [00:11<00:00, 172.66it/s]


In [434]:
K_exp_tr0 = exp_kernel(K_dist0, sigma=0.12)
K_exp_tr1 = exp_kernel(K_dist1, sigma=0.13)
K_exp_tr2 = exp_kernel(K_dist2, sigma=0.15)

print(K_exp_tr0.mean(), K_exp_tr1.mean(), K_exp_tr2.mean())
print(K_exp_tr0.std(), K_exp_tr1.std(), K_exp_tr2.std())

0.4083209343932956 0.4459201594477618 0.479113545922329
0.10634323822742853 0.11167267153460386 0.17882876159660632


### 2.2 - Local Alignment Kernel

Smith-Waterman Loacal Alignment Score  $SW$ becomes a Kernel if we take  $\widetilde{SW} = SW -\lambda_{min}(SW)I$

In [133]:
sw = sm.SmithWaterman()

In [ ]:
SW_tr0 = np.zeros((Xtr0_mat100.shape[0], Xtr0_mat100.shape[0]))

for i in tqdm(range(Xtr0_mat100.shape[0]), desc="Computing LAK_tr0"):
    for j in range(i, Xtr0_mat100.shape[0]):
        SW_tr0[i, j] = sw.get_raw_score(Xtr0['seq'].iloc[i], Xtr0['seq'].iloc[j])
        if i != j:
            SW_tr0[j, i] = SW_tr0[i, j]

SW_tr1 = np.zeros((Xtr1_mat100.shape[0], Xtr1_mat100.shape[0]))

for i in tqdm(range(Xtr1_mat100.shape[0]), desc="Computing LAK_tr1"):
    for j in range(i, Xtr1_mat100.shape[0]):
        SW_tr1[i, j] = sw.get_raw_score(Xtr1['seq'].iloc[i], Xtr1['seq'].iloc[j])
        if i != j:
            SW_tr1[j, i] = SW_tr1[i, j]

SW_tr2 = np.zeros((Xtr2_mat100.shape[0], Xtr2_mat100.shape[0]))

for i in tqdm(range(Xtr2_mat100.shape[0]), desc="Computing LAK_tr2"):
    for j in range(i, Xtr2_mat100.shape[0]):
        SW_tr2[i, j] = sw.get_raw_score(Xtr2['seq'].iloc[i], Xtr2['seq'].iloc[j])
        if i != j:
            SW_tr2[j, i] = SW_tr2[i, j]

print(SW_tr0.mean(), SW_tr1.mean(), SW_tr2.mean())
print(SW_tr0.std(), SW_tr1.std(), SW_tr2.std())

Computing LAK_tr0:  50%|█████     | 1010/2000 [11:15<14:49,  1.11it/s]

The Local Alignment Kernel defined as:
$$K_{LA}^{(\beta)}(x,y) = \sum_{\pi\in\Pi(x,y)} s_{S,g}(\pi)$$

is symmetric positive definite.

We assume an affine gap penalty:
$$\left\{\begin{aligned}
&g(0) = 0 \\
&g(n) = d + e(n-1) \quad \text{for } n>0
\end{aligned}\right.$$

where $l(\pi)$ is the length of the alignment $\pi$.

In [86]:
# Define the similarity function S (example: identity function)
def S(a, b):
    return 1 if a == b else 0

def LA_Kernel(x, y, beta, d, e):
    # Lengths of the sequences
    len_x = len(x)
    len_y = len(y)
    # Initialize matrices
    M = np.zeros((len_x + 1, len_y + 1))
    X = np.zeros((len_x + 1, len_y + 1))
    Y = np.zeros((len_x + 1, len_y + 1))
    X2 = np.zeros((len_x + 1, len_y + 1))
    Y2 = np.zeros((len_x + 1, len_y + 1))
    # Fill the matrices using the recursive equations
    for i in range(1, len_x + 1):
        for j in range(1, len_y + 1):
            # Compute M(i,j)
            M[i][j] = np.exp(beta * S(x[i-1], y[j-1])) * (1 + X[i-1][j-1] + Y[i-1][j-1] + M[i-1][j-1])
            # Compute X(i,j)
            X[i][j] = np.exp(beta * d) * M[i-1][j] + np.exp(beta * e) * X[i-1][j]            
            # Compute Y(i,j)
            Y[i][j] = np.exp(beta * d) * (M[i][j-1] + X[i][j-1]) + np.exp(beta * e) * Y[i][j-1]
            # Compute X2(i,j)
            X2[i][j] = M[i-1][j] + X2[i-1][j]            
            # Compute Y2(i,j)
            Y2[i][j] = M[i][j-1] + X2[i][j-1] + Y2[i][j-1]
    # Compute the LA kernel
    K_LA = 1 + X2[len_x][len_y] + Y2[len_x][len_y] + M[len_x][len_y]
    return K_LA

In [113]:
import numpy as np

# Define the similarity function S (example: identity function)
def S(a, b):
    return 1 if a == b else 0

def LA_Kernel_log(x, y, beta, d, e):
    # Lengths of the sequences
    len_x = len(x)
    len_y = len(y)
    
    # Initialize matrices with -inf for log(0)
    M = np.full((len_x + 1, len_y + 1), -np.inf)
    X = np.full((len_x + 1, len_y + 1), -np.inf)
    Y = np.full((len_x + 1, len_y + 1), -np.inf)
    X2 = np.full((len_x + 1, len_y + 1), -np.inf)
    Y2 = np.full((len_x + 1, len_y + 1), -np.inf)
    
    # Set log(1) = 0 at the origin
    M[0][0] = 0
    
    # Precompute constants in log-space
    log_exp_beta_d = beta * d
    log_exp_beta_e = beta * e
    
    # Fill the matrices using the recursive equations in log-space
    for i in range(1, len_x + 1):
        for j in range(1, len_y + 1):
            # Compute log(M[i][j])
            log_sum_M = np.logaddexp(0, np.logaddexp(X[i-1][j-1], np.logaddexp(Y[i-1][j-1], M[i-1][j-1])))
            M[i][j] = beta * S(x[i-1], y[j-1]) + log_sum_M
            
            # Compute log(X[i][j])
            log_X1 = log_exp_beta_d + M[i-1][j]
            log_X2 = log_exp_beta_e + X[i-1][j]
            X[i][j] = np.logaddexp(log_X1, log_X2)
            
            # Compute log(Y[i][j])
            log_Y1 = log_exp_beta_d + np.logaddexp(M[i][j-1], X[i][j-1])
            log_Y2 = log_exp_beta_e + Y[i][j-1]
            Y[i][j] = np.logaddexp(log_Y1, log_Y2)
            
            # Compute log(X2[i][j])
            X2[i][j] = np.logaddexp(M[i-1][j], X2[i-1][j])
            
            # Compute log(Y2[i][j])
            Y2[i][j] = np.logaddexp(M[i][j-1], np.logaddexp(X2[i][j-1], Y2[i][j-1]))
    
    # Compute the LA kernel in log-space
    K_LA_log = np.logaddexp(0, np.logaddexp(X2[len_x][len_y], np.logaddexp(Y2[len_x][len_y], M[len_x][len_y])))
    
    # Return the kernel value in the original space
    return np.exp(K_LA_log)

In [107]:
# Example usage:
# Define sequences and parameters
x = "AGTAC"*10
y = "AGTAC"*10
beta = .05
d = 11.0
e = 1.0

# Compute matrices and LA kernel
K_LA = LA_Kernel(x, y, beta, d, e)

print(f"LA Kernel K_LA: {K_LA:.2f}")

LA Kernel K_LA: 493229220810520924057373202121228288.00


In [ ]:
LA_tr0 = np.zeros((Xtr0_mat100.shape[0], Xtr0_mat100.shape[0]))

for i in tqdm(range(Xtr0_mat100.shape[0]), desc="Computing LA_tr0"):
    for j in range(i, Xtr0_mat100.shape[0]):
        LA_tr0[i, j] = np.log(LA_Kernel(Xtr0['seq'][i], Xtr0['seq'][j], beta, d, e)) / beta
        if i != j:
            LA_tr0[j, i] = LA_tr0[i, j]

Row 0 progress:   2%|▏         | 49/2000 [00:03<02:06, 15.37it/s]


KeyboardInterrupt: 

## 3 - Optimizer Function

In [44]:
# Define the loss function using the precomputed kernel matrix K
def loss(Y, m_t, alpha, lambd):
    n = Y.shape[0]
    loss = np.sum(np.log(1 + np.exp(-Y * m_t)))
    return lambd * np.dot(alpha, m_t) / 2 + loss / n

# Define the sigmoid function
def sigmoid(u):
    return 1/(1 + np.exp(-u))

def W_func(Y, m_t):
    n = Y.shape[0]
    W = np.zeros(n)
    W = sigmoid(m_t) * sigmoid(-m_t)
    return np.diag(W)

def P_func(Y, m_t):
    n = Y.shape[0]
    P = -sigmoid(-Y * m_t)
    return np.diag(P)

def z_func(Y, m_t):
    return m_t + Y / sigmoid(Y * m_t)

def quadratic_loss(K, Y, alpha_t, lambd, alpha):
    n = K.shape[0]
    m_t = np.dot(K, alpha_t)
    W = W_func(Y, m_t)
    P = P_func(Y, m_t)
    z = z_func(Y, m_t)
    return lambd * np.dot(alpha, np.dot(K, alpha)) + (K @ alpha - z).T @ W @ (K @ alpha - z) / n

In [50]:
# Solve KLR by IRLS on the quadratic loss
def IRLS(K, Y, alpha_0, lambd, tol=1e-6, max_iter=100, Y_test=None, K_test=None):
    n = K.shape[0]
    alpha_t = alpha_0.copy()
    criterion = np.inf
    for i in tqdm(range(max_iter), desc="IRLS Progress"):
        m_t = np.dot(K, alpha_t)
        accuracy = np.mean(np.sign(m_t) == Y)
        if Y_test is not None and K_test is not None:
            test_accuracy = np.mean(np.sign(np.dot(K_test, alpha_t)) == Y_test)
            print(f"Iteration {i}: Loss - {loss(Y, m_t, alpha_t, lambd):.3f}, Convergence Criterion - : {criterion}, \n Train Accuracy - {accuracy:.2f}, Test Accuracy - {test_accuracy}")
        else:
            print(f"Iteration {i}: Loss - {loss(Y, m_t, alpha_t, lambd):.3f}, Convergence Criterion - : {criterion}, Train Accuracy - {accuracy}")
        W_t = W_func(Y, m_t)
        z_t = z_func(Y, m_t)
        W_t_sqrt = np.sqrt(W_t)
        A = (W_t_sqrt @ K @ W_t_sqrt + lambd * n * np.eye(n)) @ np.diag(1 / np.diag(W_t_sqrt))
        B = W_t_sqrt @ z_t
        alpha_new = np.linalg.solve(A, B)
        criterion = np.linalg.norm(alpha_new - alpha_t)
        if criterion < tol:
            accuracy = np.mean(np.sign(np.dot(K, alpha_new)) == Y)
            if Y_test is not None and K_test is not None:
                test_accuracy = np.mean(np.sign(np.dot(K_test, alpha_new)) == Y_test)
                print(f"END : Iteration {i}: Loss - {loss(Y, m_t, alpha_t, lambd):.3f}, Convergence Criterion - : {criterion}, \n Train Accuracy - {accuracy:.2f}, Test Accuracy - {test_accuracy}")
            else:
                print(f"END : Iteration {i}: Loss - {loss(Y, m_t, alpha_t, lambd):.3f}, Convergence Criterion - : {criterion}, Train Accuracy - {accuracy}")
            break
        alpha_t = alpha_new
    return alpha_t

## 4 - Train & Test Model

In [73]:
SW_tr0 = np.load('SW_tr0.npy')
# If the smallest eigenvalue is negative, shift the matrix to make it positive definite
eigenvalues, _ = np.linalg.eigh(SW_tr0)
min_eigenvalue = np.min(eigenvalues)
if min_eigenvalue < 0:
    SW_tr0 += np.eye(SW_tr0.shape[0]) * (-min_eigenvalue + 1e-6)

In [ ]:
n_train = 1600

def split_kernel_matrix(K_exp, n_train):
    K_train = K_exp[:n_train, :n_train]
    K_test = K_exp[n_train:, :n_train]
    return K_train, K_test

# Split the datasets into training and testing sets
K_tr0_tain, K_tr0_test = split_kernel_matrix(SW_tr0, n_train)
K_tr1_train, K_tr1_test = split_kernel_matrix(SW_tr1, n_train)
K_tr2_train, K_tr2_test = split_kernel_matrix(SW_tr2, n_train)

Ytr0_train, Ytr0_test = Ytr0[:n_train], Ytr0[n_train:]
Ytr1_train, Ytr1_test = Ytr1[:n_train], Ytr1[n_train:]
Ytr2_train, Ytr2_test = Ytr2[:n_train], Ytr2[n_train:]

In [ ]:
lambd_0 = 0.1
alpha_0_0 = np.zeros(K_tr0_tain.shape[0])
# Run the IRLS algorithm with the new initialization for the first dataset
alpha_0_0 = IRLS(K_tr0_tain, Ytr0_train, alpha_0_0, lambd_0, 
                 tol=1e-5, max_iter=10, 
                 Y_test=Ytr0_test, K_test=K_tr0_test)
print(f'Alpha norm : {np.linalg.norm(alpha_0_0):.2f}')

IRLS Progress:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration 0: Loss - 0.693, Convergence Criterion - : inf, 
 Train Accuracy - 0.00, Test Accuracy - 0.0


IRLS Progress:  10%|█         | 1/10 [00:01<00:15,  1.67s/it]

Iteration 1: Loss - 0.658, Convergence Criterion - : 0.11753946248495575, 
 Train Accuracy - 0.69, Test Accuracy - 0.585


IRLS Progress:  20%|██        | 2/10 [00:05<00:24,  3.03s/it]

Iteration 2: Loss - 0.658, Convergence Criterion - : 0.0004802193847646569, 
 Train Accuracy - 0.69, Test Accuracy - 0.5875


IRLS Progress:  20%|██        | 2/10 [00:08<00:34,  4.31s/it]

END : Iteration 2: Loss - 0.658, Convergence Criterion - : 4.6153682957069054e-07, 
 Train Accuracy - 0.69, Test Accuracy - 0.5875
Alpha norm : 0.12


In [420]:
lambd_1 = 0.0001
alpha_0_1 = np.zeros(K_exp_tr1_train.shape[0])
# Run the IRLS algorithm with the new initialization for the second dataset
alpha_0_1 = IRLS(K_exp_tr1_train, Ytr1_train, alpha_0_1, lambd_1, 
                 tol=1e-5, max_iter=10, 
                 Y_test=Ytr1_test, K_test=K_exp_tr1_test)

IRLS Progress:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration 0: Loss - 0.1848392481493187, Convergence Criterion - : inf, 
 Train Accuracy - 0.0, Test Accuracy - 0.0


IRLS Progress:  10%|█         | 1/10 [00:01<00:14,  1.63s/it]

Iteration 1: Loss - 0.19274529091606862, Convergence Criterion - : 90.55132484007582, 
 Train Accuracy - 0.878125, Test Accuracy - 0.63


IRLS Progress:  20%|██        | 2/10 [00:03<00:14,  1.76s/it]

...

Iteration 3: Loss - 0.19709953036609476, Convergence Criterion - : 0.19142091382917892, 
 Train Accuracy - 0.86, Test Accuracy - 0.6225


IRLS Progress:  40%|████      | 4/10 [00:11<00:20,  3.42s/it]

Iteration 4: Loss - 0.19709988098664294, Convergence Criterion - : 0.0004643480766235943, 
 Train Accuracy - 0.86, Test Accuracy - 0.6225


IRLS Progress:  40%|████      | 4/10 [00:16<00:24,  4.04s/it]

END : Iteration 4: Loss - 0.19709988098664294, Convergence Criterion - : 3.5436569159146845e-09, 
 Train Accuracy - 0.86, Test Accuracy - 0.6225


In [421]:
lambd_2 = 0.001
alpha_0_2 = np.zeros(K_exp_tr2_train.shape[0])
# Run the IRLS algorithm with the new initialization for the third dataset
alpha_0_2 = IRLS(K_exp_tr2_train, Ytr2_train, alpha_0_2, lambd_2, 
                 tol=1e-5, max_iter=10, 
                 Y_test=Ytr2_test, K_test=K_exp_tr2_test)

IRLS Progress:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration 0: Loss - 0.1848392481493187, Convergence Criterion - : inf, 
 Train Accuracy - 0.0, Test Accuracy - 0.0


IRLS Progress:  10%|█         | 1/10 [00:02<00:19,  2.14s/it]

Iteration 1: Loss - 0.18402103376220147, Convergence Criterion - : 11.40854893028931, 
 Train Accuracy - 0.68125, Test Accuracy - 0.6275


IRLS Progress:  20%|██        | 2/10 [00:05<00:21,  2.63s/it]

...

Iteration 3: Loss - 0.1855452987762537, Convergence Criterion - : 0.019180073683026006, 
 Train Accuracy - 0.683125, Test Accuracy - 0.64


IRLS Progress:  40%|████      | 4/10 [00:14<00:23,  3.97s/it]

Iteration 4: Loss - 0.18554706682864164, Convergence Criterion - : 0.0002464729987810541, 
 Train Accuracy - 0.683125, Test Accuracy - 0.64


IRLS Progress:  40%|████      | 4/10 [00:18<00:27,  4.54s/it]

END : Iteration 4: Loss - 0.18554706682864164, Convergence Criterion - : 3.99055653608917e-08, 
 Train Accuracy - 0.683125, Test Accuracy - 0.64


In [457]:
def calculate_accuracy(K_train, K_test, Y_train, Y_test, alpha):
    train_predictions = np.sign(K_train @ alpha)
    train_accuracy = np.mean(train_predictions == Y_train) * 100
    print(f"Train Accuracy: {train_accuracy:.2f}%")

    test_predictions = np.sign(K_test @ alpha)
    test_accuracy = np.mean(test_predictions == Y_test) * 100
    print(f"Test Accuracy: {test_accuracy:.2f}%")

    print(f"Alpha's norm: {np.linalg.norm(alpha):.2f}")
    return train_accuracy, test_accuracy

# Calculate accuracy for the first dataset
print("-----First dataset-----")
_, a0 = calculate_accuracy(K_exp_tr0_train, K_exp_tr0_test, Ytr0_train, Ytr0_test, alpha_0_0)
# Calculate accuracy for the second dataset
print("-----Second dataset-----")
_, a1 = calculate_accuracy(K_exp_tr1_train, K_exp_tr1_test, Ytr1_train, Ytr1_test, alpha_0_1)
# Calculate accuracy for the third dataset
print("-----Third dataset-----")
_, a2 = calculate_accuracy(K_exp_tr2_train, K_exp_tr2_test, Ytr2_train, Ytr2_test, alpha_0_2)

# Mean accuracy
mean_accuracy = (a0 + a1 + a2) / 3
print(f"Mean accuracy: {mean_accuracy:.2f}%")

-----First dataset-----
Train Accuracy: 87.44%
Test Accuracy: 58.75%
Alpha's norm: 94.13
-----Second dataset-----
Train Accuracy: 86.00%
Test Accuracy: 62.25%
Alpha's norm: 91.84
-----Third dataset-----
Train Accuracy: 68.31%
Test Accuracy: 64.00%
Alpha's norm: 11.43
Mean accuracy: 61.67%


## Evaluate on Test Set

In [431]:
# Load the matrix representation of the sequences
Xte0_mat100 = pd.read_csv('Xte0_mat100.csv', header=None, sep=' ').values
Xte1_mat100 = pd.read_csv('Xte1_mat100.csv', header=None, sep=' ').values
Xte2_mat100 = pd.read_csv('Xte2_mat100.csv', header=None, sep=' ').values

In [435]:
# Initialize the distance matrices for test data
K_dist_te0 = np.zeros((Xte0_mat100.shape[0], n_train))
K_dist_te1 = np.zeros((Xte1_mat100.shape[0], n_train))
K_dist_te2 = np.zeros((Xte2_mat100.shape[0], n_train))

# Compute the distance matrices for test data
for i in tqdm(range(Xte0_mat100.shape[0]), desc="Computing K_dist_te0"):
    for j in range(n_train):
        K_dist_te0[i, j] = dist_kernel(Xte0_mat100[i], Xtr0_mat100[j])

for i in tqdm(range(Xte1_mat100.shape[0]), desc="Computing K_dist_te1"):
    for j in range(n_train):
        K_dist_te1[i, j] = dist_kernel(Xte1_mat100[i], Xtr1_mat100[j])

for i in tqdm(range(Xte2_mat100.shape[0]), desc="Computing K_dist_te2"):
    for j in range(n_train):
        K_dist_te2[i, j] = dist_kernel(Xte2_mat100[i], Xtr2_mat100[j])

print(K_dist_te0.mean(), K_dist_te1.mean(), K_dist_te2.mean())
print(K_dist_te0.std(), K_dist_te1.std(), K_dist_te2.std())

Computing K_dist_te2: 100%|██████████| 1000/1000 [00:04<00:00, 221.37it/s]

0.16318112129091467 0.1667362036036887 0.1911287852576293
0.02754851287049635 0.03005545310859116 0.06623979367512123


In [436]:
K_exp_te0 = exp_kernel(K_dist_te0, sigma=0.12)
K_exp_te1 = exp_kernel(K_dist_te1, sigma=0.13)
K_exp_te2 = exp_kernel(K_dist_te2, sigma=0.15)

print(K_exp_te0.mean(), K_exp_te1.mean(), K_exp_te2.mean())
print(K_exp_te0.std(), K_exp_te1.std(), K_exp_te2.std())

0.40591689794208635 0.4479003039324682 0.4761975265751476
0.10760827434663765 0.11088148276263982 0.18260396395836956


In [437]:
# Predictions
Yte0 = np.sign(K_exp_te0 @ alpha_0_0)
Yte1 = np.sign(K_exp_te1 @ alpha_0_1)
Yte2 = np.sign(K_exp_te2 @ alpha_0_2)

In [453]:
# Concatenate and add Id column
Yte = np.concatenate([Yte0, Yte1, Yte2])
Yte = (Yte + 1) // 2
Yte = pd.DataFrame(data=Yte, columns=['Bound'], dtype='int64')
Yte.insert(0, 'Id', Yte.index)

In [454]:
Yte.to_csv('Yte.csv', index=False)